In [1]:
from PIL import Image
import torch
from tqdm.auto import tqdm

from point_e.diffusion.configs import DIFFUSION_CONFIGS, diffusion_from_config
from point_e.diffusion.sampler import PointCloudSampler
from point_e.models.download import load_checkpoint
from point_e.models.configs import MODEL_CONFIGS, model_from_config
from point_e.util.plotting import plot_point_cloud

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('creating base model...')
base_name = 'base1B' #'base40M' # use base300M or base1B for better results
base_model = model_from_config(MODEL_CONFIGS[base_name], device)
base_model.eval()
base_diffusion = diffusion_from_config(DIFFUSION_CONFIGS[base_name])

print('creating upsample model...')
upsampler_model = model_from_config(MODEL_CONFIGS['upsample'], device)
upsampler_model.eval()
upsampler_diffusion = diffusion_from_config(DIFFUSION_CONFIGS['upsample'])

print('downloading base checkpoint...')
base_model.load_state_dict(load_checkpoint(base_name, device))

print('downloading upsampler checkpoint...')
upsampler_model.load_state_dict(load_checkpoint('upsample', device))

creating base model...
creating upsample model...
downloading base checkpoint...
downloading upsampler checkpoint...


<All keys matched successfully>

In [3]:
sampler = PointCloudSampler(
    device=device,
    models=[base_model, upsampler_model],
    diffusions=[base_diffusion, upsampler_diffusion],
    num_points=[1024, 4096 - 1024],
    aux_channels=['R', 'G', 'B'],
    guidance_scale=[3.0, 3.0],
)

# FULL RUN

In [ ]:
# Load an image to condition on.
# img = Image.open('example_data/cube_stack.jpg')
from pathlib import Path
file_paths = [file_path for file_path in Path('firework_example_data/30-ts-2_serpent').glob('*.png')]

for path in file_paths:
    img = Image.open(str(path))

    # Produce a sample from the model.
    samples = None
    for x in tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(images=[img]))):
        samples = x
    pc = sampler.output_to_point_clouds(samples)[0]
    pc.save(f'firework_example_data/pc_{path.stem}.npz')

# SINGLE RUN

In [ ]:
# Load an image to condition on.
# img = Image.open('example_data/cube_stack.jpg')
from pathlib import Path
path = Path('firework_example_data/30-ts-2_serpent/012.png')

img = Image.open(str(path))

# Produce a sample from the model.
samples = None
for x in tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(images=[img]))):
    samples = x
pc = sampler.output_to_point_clouds(samples)[0]
pc.save(f'firework_example_data/pc_{path.stem}.npz')

In [ ]:
pc = sampler.output_to_point_clouds(samples)[0]
fig = plot_point_cloud(pc, grid_size=3, fixed_bounds=((-0.75, -0.75, -0.75),(0.75, 0.75, 0.75)))

In [ ]:
pc.save('example_data/pc_012.npz')

# check out pointcloud

In [25]:
from point_e.util.point_cloud import PointCloud
pc = PointCloud.load('example_data/pc_012.npz')

In [2]:

%matplotlib notebook

import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import numpy as np


fixed_bounds = ((-0.75, -0.75, -0.75),
                (0.75, 0.75, 0.75))
color = True

sns.set(style = "darkgrid")
fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(1, 1, 1, projection="3d")
color_args = {}
if color:
    color_args["c"] = np.stack(
        [pc.channels["R"], pc.channels["G"], pc.channels["B"]], axis=-1
    )
c = pc.coords

ax.scatter(c[:, 0], c[:, 1], c[:, 2], **color_args)

ax.set_xlim3d(fixed_bounds[0][0], fixed_bounds[1][0])
ax.set_ylim3d(fixed_bounds[0][1], fixed_bounds[1][1])
ax.set_zlim3d(fixed_bounds[0][2], fixed_bounds[1][2])

plt.show()

<IPython.core.display.Javascript object>

In [1]:
from point_e.util.point_cloud import PointCloud
import torch
import kaolin

# arguments and hyperparameters
device = 'cuda'
lr = 1e-3
laplacian_weight = 0.8
iterations = 10000
save_every = 100
multires = 8
grid_res = 128

pc_data = 'example_data/pc_012.npz'
#pc_data = 'example_data/pc_corgi.npz'
pc = PointCloud.load(pc_data)
points = pc.coords
#pcd_path = "/home/ubuntu/point-e/point_e/util/dmtet/usd/bear_pointcloud.usd"
#points = kaolin.io.usd.import_pointclouds(pcd_path)[0].points.to(device)

center = (points.max(0)[0] + points.min(0)[0]) / 2
max_l = (points.max(0)[0] - points.min(0)[0]).max()
points = ((points - center) / max_l)* 0.9
points = torch.tensor(points).to(device)

In [2]:
from point_e.util.dmtet.dmtet_network import Decoder
import torch
import numpy as np

tet_verts = torch.tensor(np.load('/home/ubuntu/point-e/point_e/util/dmtet/samples/{}_verts.npz'.format(grid_res))['data'], dtype=torch.float, device=device)
tets = torch.tensor(([np.load('/home/ubuntu/point-e/point_e/util/dmtet/samples/{}_tets_{}.npz'.format(grid_res, i))['data'] for i in range(4)]), dtype=torch.long, device=device).permute(1,0)
print(tet_verts.shape, tets.shape)

# Initialize model and create optimizer
model = Decoder(multires=multires).to(device)
model.pre_train_sphere(1000)

/tmp/ipykernel_2702681/4210860135.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  tets = torch.tensor(([np.load('/home/ubuntu/point-e/point_e/util/dmtet/samples/{}_tets_{}.npz'.format(grid_res, i))['data'] for i in range(4)]), dtype=torch.long, device=device).permute(1,0)


torch.Size([277410, 3]) torch.Size([1524684, 4])
Initialize SDF to sphere


100%|██████████| 1000/1000 [00:02<00:00, 403.67it/s]

Pre-trained MLP 9.816072270041332e-05


In [3]:
# Laplacian regularization using umbrella operator (Fujiwara / Desbrun).
# https://mgarland.org/class/geom04/material/smoothing.pdf
import kaolin

def laplace_regularizer_const(mesh_verts, mesh_faces):
    term = torch.zeros_like(mesh_verts)
    norm = torch.zeros_like(mesh_verts[..., 0:1])

    v0 = mesh_verts[mesh_faces[:, 0], :]
    v1 = mesh_verts[mesh_faces[:, 1], :]
    v2 = mesh_verts[mesh_faces[:, 2], :]

    term.scatter_add_(0, mesh_faces[:, 0:1].repeat(1,3), (v1 - v0) + (v2 - v0))
    term.scatter_add_(0, mesh_faces[:, 1:2].repeat(1,3), (v0 - v1) + (v2 - v1))
    term.scatter_add_(0, mesh_faces[:, 2:3].repeat(1,3), (v0 - v2) + (v1 - v2))

    two = torch.ones_like(v0) * 2.0
    norm.scatter_add_(0, mesh_faces[:, 0:1], two)
    norm.scatter_add_(0, mesh_faces[:, 1:2], two)
    norm.scatter_add_(0, mesh_faces[:, 2:3], two)

    term = term / torch.clamp(norm, min=1.0)

    return torch.mean(term**2)

def loss_f(mesh_verts, mesh_faces, points, it):
    pred_points = kaolin.ops.mesh.sample_points(mesh_verts.unsqueeze(0), mesh_faces, 50000)[0][0]
    chamfer = kaolin.metrics.pointcloud.chamfer_distance(pred_points.unsqueeze(0), points.unsqueeze(0)).mean()
    if it > iterations//2:
        lap = laplace_regularizer_const(mesh_verts, mesh_faces)
        return chamfer + lap * laplacian_weight
    return chamfer

In [4]:
vars = [p for _, p in model.named_parameters()]
optimizer = torch.optim.Adam(vars, lr=lr)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time

In [5]:
logs_path = './logs'

# We initialize the timelapse that will store USD for the visualization apps
timelapse = kaolin.visualize.Timelapse(logs_path)

In [6]:
for it in range(iterations):
    pred = model(tet_verts) # predict SDF and per-vertex deformation
    sdf, deform = pred[:,0], pred[:,1:]
    verts_deformed = tet_verts + torch.tanh(deform) / grid_res # constraint deformation to avoid flipping tets
    mesh_verts, mesh_faces = kaolin.ops.conversions.marching_tetrahedra(verts_deformed.unsqueeze(0), tets, sdf.unsqueeze(0)) # running MT (batched) to extract surface mesh
    #print(mesh_verts, mesh_faces)
    mesh_verts, mesh_faces = mesh_verts[0], mesh_faces[0]

    loss = loss_f(mesh_verts, mesh_faces, points, it)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()
    if (it) % save_every == 0 or it == (iterations - 1): 
        print ('Iteration {} - loss: {}, # of mesh vertices: {}, # of mesh faces: {}'.format(it, loss, mesh_verts.shape[0], mesh_faces.shape[0]))
        # save reconstructed mesh
        timelapse.add_mesh_batch(
            iteration=it+1,
            category='extracted_mesh',
            vertices_list=[mesh_verts.cpu()],
            faces_list=[mesh_faces.cpu()]
        )

Iteration 0 - loss: 0.039039406925439835, # of mesh vertices: 25728, # of mesh faces: 51444
Iteration 100 - loss: 0.005376047920435667, # of mesh vertices: 60715, # of mesh faces: 119768
Iteration 200 - loss: 0.06702491641044617, # of mesh vertices: 365881, # of mesh faces: 725873
Iteration 300 - loss: 0.07136605679988861, # of mesh vertices: 347702, # of mesh faces: 684706
Iteration 400 - loss: 0.006381798069924116, # of mesh vertices: 64075, # of mesh faces: 127338
Iteration 500 - loss: 0.0027827471494674683, # of mesh vertices: 43988, # of mesh faces: 87634
Iteration 600 - loss: 0.0016419888706877828, # of mesh vertices: 42951, # of mesh faces: 85750
Iteration 700 - loss: 0.0006066123605705798, # of mesh vertices: 30706, # of mesh faces: 61458
Iteration 800 - loss: 0.0006712912581861019, # of mesh vertices: 31250, # of mesh faces: 62560
Iteration 900 - loss: 0.000875496247317642, # of mesh vertices: 34092, # of mesh faces: 68053
Iteration 1000 - loss: 0.0007283350569196045, # of mes

In [33]:
points.shape

torch.Size([4096, 3])

In [18]:
from point_e.util.point_cloud import PointCloud
pc = PointCloud.load('example_data/pc_012.npz')
torch.tensor(pc.coords)

tensor([[ 0.1501,  0.1312, -0.0160],
        [ 0.0748,  0.1856, -0.0396],
        [ 0.1481,  0.3223, -0.0360],
        ...,
        [ 0.3628,  0.2961, -0.0100],
        [ 0.2895,  0.2162,  0.0100],
        [-0.0581,  0.1527,  0.0432]])

# eliminate KAOLIN library

In [1]:
from point_e.util.point_cloud import PointCloud
import torch

# arguments and hyperparameters
device = 'cuda'
lr = 1e-3
laplacian_weight = 0.8
iterations = 10000
save_every = 100
multires = 8
grid_res = 128

pc_data = 'example_data/pc_012.npz'
#pc_data = 'example_data/pc_corgi.npz'
pc = PointCloud.load(pc_data)
points = pc.coords
#pcd_path = "/home/ubuntu/point-e/point_e/util/dmtet/usd/bear_pointcloud.usd"
#points = kaolin.io.usd.import_pointclouds(pcd_path)[0].points.to(device)

center = (points.max(0)[0] + points.min(0)[0]) / 2
max_l = (points.max(0)[0] - points.min(0)[0]).max()
points = ((points - center) / max_l)* 0.9
points = torch.tensor(points).to(device)

In [2]:
from point_e.util.dmtet.dmtet_network import Decoder
import torch
import numpy as np

tet_verts = torch.tensor(np.load('/home/ubuntu/point-e/point_e/util/dmtet/samples/{}_verts.npz'.format(grid_res))['data'], dtype=torch.float, device=device)
tets = torch.tensor(([np.load('/home/ubuntu/point-e/point_e/util/dmtet/samples/{}_tets_{}.npz'.format(grid_res, i))['data'] for i in range(4)]), dtype=torch.long, device=device).permute(1,0)
print(tet_verts.shape, tets.shape)

# Initialize model and create optimizer
model = Decoder(multires=multires).to(device)
model.pre_train_sphere(1000)

ImportError: dynamic module does not define module export function (PyInit__C)

In [3]:
# Laplacian regularization using umbrella operator (Fujiwara / Desbrun).
# https://mgarland.org/class/geom04/material/smoothing.pdf
import kaolin
import point_e

def laplace_regularizer_const(mesh_verts, mesh_faces):
    term = torch.zeros_like(mesh_verts)
    norm = torch.zeros_like(mesh_verts[..., 0:1])

    v0 = mesh_verts[mesh_faces[:, 0], :]
    v1 = mesh_verts[mesh_faces[:, 1], :]
    v2 = mesh_verts[mesh_faces[:, 2], :]

    term.scatter_add_(0, mesh_faces[:, 0:1].repeat(1,3), (v1 - v0) + (v2 - v0))
    term.scatter_add_(0, mesh_faces[:, 1:2].repeat(1,3), (v0 - v1) + (v2 - v1))
    term.scatter_add_(0, mesh_faces[:, 2:3].repeat(1,3), (v0 - v2) + (v1 - v2))

    two = torch.ones_like(v0) * 2.0
    norm.scatter_add_(0, mesh_faces[:, 0:1], two)
    norm.scatter_add_(0, mesh_faces[:, 1:2], two)
    norm.scatter_add_(0, mesh_faces[:, 2:3], two)

    term = term / torch.clamp(norm, min=1.0)

    return torch.mean(term**2)

def loss_f(mesh_verts, mesh_faces, points, it):
    pred_points = point_e.util.dmtet.trianglemesh.sample_points(mesh_verts.unsqueeze(0), mesh_faces, 50000)[0][0]
    chamfer = point_e.util.dmtet.pointcloud.chamfer_distance(pred_points.unsqueeze(0), points.unsqueeze(0)).mean()
    if it > iterations//2:
        lap = laplace_regularizer_const(mesh_verts, mesh_faces)
        return chamfer + lap * laplacian_weight
    return chamfer

In [4]:
vars = [p for _, p in model.named_parameters()]
optimizer = torch.optim.Adam(vars, lr=lr)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time

logs_path = './logs'

# We initialize the timelapse that will store USD for the visualization apps
#timelapse = kaolin.visualize.Timelapse(logs_path)

NameError: name 'model' is not defined

In [5]:
for it in range(iterations):
    pred = model(tet_verts) # predict SDF and per-vertex deformation
    sdf, deform = pred[:,0], pred[:,1:]
    verts_deformed = tet_verts + torch.tanh(deform) / grid_res # constraint deformation to avoid flipping tets
    mesh_verts, mesh_faces = point_e.util.dmtet.tetmesh.marching_tetrahedra(verts_deformed.unsqueeze(0), tets, sdf.unsqueeze(0)) # running MT (batched) to extract surface mesh
    #print(mesh_verts, mesh_faces)
    mesh_verts, mesh_faces = mesh_verts[0], mesh_faces[0]

    loss = loss_f(mesh_verts, mesh_faces, points, it)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()
    if (it) % save_every == 0 or it == (iterations - 1): 
        print ('Iteration {} - loss: {}, # of mesh vertices: {}, # of mesh faces: {}'.format(it, loss, mesh_verts.shape[0], mesh_faces.shape[0]))
        # save reconstructed mesh
        # timelapse.add_mesh_batch(
        #     iteration=it+1,
        #     category='extracted_mesh',
        #     vertices_list=[mesh_verts.cpu()],
        #     faces_list=[mesh_faces.cpu()]
        # )

NameError: name 'model' is not defined